In [1]:
import os
os.chdir('../../' )
cwd=os.getcwd()

from oligo_designer_toolsuite.oligo_specificity_filter._filter_base import ProbeFilterBase
from oligo_designer_toolsuite.oligo_specificity_filter._filter_exact import ProbeFilterExact
from oligo_designer_toolsuite.oligo_specificity_filter._filter_blastn import ProbeFilterBlastn

In [2]:
cwd

'/home/chels/CRISPR/repos/oligo-designer-toolsuite'

In [2]:
number_batches=10
ligation_region=0
dir_output=cwd + "/output/"
dir_annotations=dir_output+"annotations/"
file_probe_info=dir_annotations + "oligo_DB_unknown_unknown_Custom_release_unknown_gene_transcript"
genes=genes = ['WASH7P', 'DDX11L1', 'TRNT', 'NOC2L', 'PLEKHN1', 'AGRN','UBE2J2', 'DVL1', 'MIB2', 'LOC112268402_1' ]
min_probes_per_gene=2


Test exact match filter

In [3]:
filter_exact=ProbeFilterExact(number_batches, ligation_region,
        dir_output,
        file_probe_info,
        genes,
        min_probes_per_gene,
        dir_annotations)

In [4]:
filter_exact.apply()

Test Blast filter

In [5]:
file_transcriptome_fasta=dir_output +"annotations/reference_DB_unknown_unknown_Custom_release_unknown_genome_False_gene_transcript_True"
word_size=10
percent_identity=80
probe_length_min=30
probe_length_max=40
coverage=50

In [6]:
blast_filter=ProbeFilterBlastn(number_batches,
        file_transcriptome_fasta, 
        ligation_region,
        dir_output,
        file_probe_info,
        genes,
        min_probes_per_gene, word_size, percent_identity, probe_length_min, probe_length_max, coverage)

In [7]:
blast_filter.apply()